In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from mongo import attraction_db
from model.google import Google
from model.Word import Word
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
import random
import os
import json
import time
import csv
import os

c:\2_DataScience\04_google_map_crawler\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [81]:
url = 'https://www.google.com/maps/search/%E6%99%AF%E9%BB%9E'

driver = webdriver.Chrome()
driver.get(url)

In [3]:
def getPage():
    page_content = driver.page_source
    # 將 HTML 內容轉換成 BeautifulSoup 物件，html.parser 為使用的解析器
    soup = BeautifulSoup(page_content, 'html.parser')
    # 透過 select 使用 CSS 選擇器 選取我們要選的 html 內容
    return soup.select('.Nv2PK.THOPZb.CpccDe')

def scrollPage(times):   # 滾動頁面
    counter = 0
    while counter <= times:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        counter += 1

def getElement():  # 抓取目前的所有留言
    spots_list = []
    scrollPage(5)
    elements = getPage()

    for element in elements:
        name = element.select('.qBF1Pd.fontHeadlineSmall')[0].text
        url = element.select('.hfpxzc')[0].get('href')
        dataset = {}
        dataset['name'] = name
        dataset['google_url'] = url

        spots_list.append(dataset)
    return spots_list

In [297]:
lastSpot = ''
while 7 == 7:
    print(lastSpot)
    spots_list = getElement()
    if (spots_list[len(spots_list)-1]["name"] == lastSpot ):
        break
    lastSpot = spots_list[len(spots_list)-1]["name"]
    time.sleep(1)
    getElement()

csv_file = './data/spot_list/spots.csv'
if os.path.exists(csv_file):
    with open(csv_file, mode="r", newline="", encoding="utf-8-sig") as file:
        reader = csv.DictReader(file)
        existing_data = list(reader)
else:
    existing_data = []

for spot in spots_list:
    existing_data.append(spot)

print(len(existing_data))


with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    # 定義 CSV 欄位
    fieldnames = ["name", "google_url"]
    # 建立 CSV 寫入器
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

    print(f"已將資料寫入 CSV 檔案：{csv_file}")


後浦16藝文特區
金水學校展示館
清遠湖
7611
已將資料寫入 CSV 檔案：./data/spot_list/spots.csv


In [26]:
dataset = pd.read_csv('./data/spot_list/spotsList_clean.csv')
dataset

,name,google_url
0,精英溫泉,https://www.google.com/maps/place/%E7%B2%BE%E8...
1,龍鳳海灘,https://www.google.com/maps/place/%E9%BE%8D%E9...
2,七星潭海岸風景特定區,https://www.google.com/maps/place/%E4%B8%83%E6...
3,謙桂緣,https://www.google.com/maps/place/%E8%AC%99%E6...
4,林口．樂活公園/捷運主題公園,https://www.google.com/maps/place/%E6%9E%97%E5...
...,...,...
1886,錫安山,https://www.google.com/maps/place/%E9%8C%AB%E5...
1887,朴子藝術公園,https://www.google.com/maps/place/%E6%9C%B4%E5...
1888,埤頭木棉花道,https://www.google.com/maps/place/%E5%9F%A4%E9...
1889,福壽山農場｜千櫻園,https://www.google.com/maps/place/%E7%A6%8F%E5...


In [28]:
df_unique = dataset.drop_duplicates(subset="name", keep="first")
df_unique.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")
df_unique

,name,google_url
0,精英溫泉,https://www.google.com/maps/place/%E7%B2%BE%E8...
1,龍鳳海灘,https://www.google.com/maps/place/%E9%BE%8D%E9...
2,七星潭海岸風景特定區,https://www.google.com/maps/place/%E4%B8%83%E6...
3,謙桂緣,https://www.google.com/maps/place/%E8%AC%99%E6...
4,林口．樂活公園/捷運主題公園,https://www.google.com/maps/place/%E6%9E%97%E5...
...,...,...
1886,錫安山,https://www.google.com/maps/place/%E9%8C%AB%E5...
1887,朴子藝術公園,https://www.google.com/maps/place/%E6%9C%B4%E5...
1888,埤頭木棉花道,https://www.google.com/maps/place/%E5%9F%A4%E9...
1889,福壽山農場｜千櫻園,https://www.google.com/maps/place/%E7%A6%8F%E5...


In [312]:
existing_names = attraction_db.attractionInfo.distinct("name")
df_filtered = df_unique[~df_unique["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")

In [24]:
existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/random_spotsList.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList_clean.csv', index=False, encoding="utf-8-sig")
df_filtered

,name,google_url
1,精英溫泉,https://www.google.com/maps/place/%E7%B2%BE%E8...
27,龍鳳海灘,https://www.google.com/maps/place/%E9%BE%8D%E9...
50,七星潭海岸風景特定區,https://www.google.com/maps/place/%E4%B8%83%E6...
67,謙桂緣,https://www.google.com/maps/place/%E8%AC%99%E6...
86,林口．樂活公園/捷運主題公園,https://www.google.com/maps/place/%E6%9E%97%E5...
...,...,...
2396,錫安山,https://www.google.com/maps/place/%E9%8C%AB%E5...
2397,朴子藝術公園,https://www.google.com/maps/place/%E6%9C%B4%E5...
2398,埤頭木棉花道,https://www.google.com/maps/place/%E5%9F%A4%E9...
2399,福壽山農場｜千櫻園,https://www.google.com/maps/place/%E7%A6%8F%E5...


In [43]:
# ============================================
# TEST
# ============================================

spotsList = pd.read_csv('./data/spot_list/spotsList_clean.csv') 

for index, row in spotsList[558:570].iterrows():
    print(index,row['name'],row['google_url'])

558 吳福洋襪子故事館 https://www.google.com/maps/place/%E5%90%B3%E7%A6%8F%E6%B4%8B%E8%A5%AA%E5%AD%90%E6%95%85%E4%BA%8B%E9%A4%A8/data=!4m7!3m6!1s0x3442a65822bf02c1:0x8f746bb3eb0ab740!8m2!3d25.0758571!4d121.401368!16s%2Fg%2F11bbrlzcgq!19sChIJwQK_IlimQjQRQLcK67NrdI8?authuser=0&hl=zh-TW&rclk=1
559 福德坑環保復育公園 https://www.google.com/maps/place/%E7%A6%8F%E5%BE%B7%E5%9D%91%E7%92%B0%E4%BF%9D%E5%BE%A9%E8%82%B2%E5%85%AC%E5%9C%92/data=!4m7!3m6!1s0x3442aafa9cc0f013:0x7d9fb57eb92a1c9b!8m2!3d25.0089367!4d121.5905109!16s%2Fg%2F12h2syq03!19sChIJE_DAnPqqQjQRmxwquX61n30?authuser=0&hl=zh-TW&rclk=1
560 八角瞭望台 https://www.google.com/maps/place/%E5%85%AB%E8%A7%92%E7%9E%AD%E6%9C%9B%E5%8F%B0/data=!4m7!3m6!1s0x3467f746ee9f22bb:0x4d15bbdd815532af!8m2!3d24.850707!4d121.8295889!16s%2Fg%2F11f10p6znr!19sChIJuyKf7kb3ZzQRrzJVgd27FU0?authuser=0&hl=zh-TW&rclk=1
561 臺鹽通霄觀光園區 https://www.google.com/maps/place/%E8%87%BA%E9%B9%BD%E9%80%9A%E9%9C%84%E8%A7%80%E5%85%89%E5%9C%92%E5%8D%80/data=!4m7!3m6!1s0x3469a624e5cf658b:0xf698f878205b51

In [5]:
# 取得 Google URL

name_to_url = {}
with open('./data/spot_list/spotsList_clean.csv', 'r', encoding='utf-8') as spots_file:
    spots_reader = csv.reader(spots_file)
    next(spots_reader)  # 跳过标题行
    for row in spots_reader:
        name = row[0]  # 假设名称在第一列
        url = row[1]   # 假设URL在第二列
        name_to_url[name] = url

def get_url_for_name(name):
    return name_to_url.get(name, "URL not found")

In [66]:
# 含有多少沒有抓

existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/spotsList_clean.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered

,name,google_url
8,真笠山東南峰,https://www.google.com/maps/place/%E7%9C%9F%E7...
16,明池國家森林遊樂區,https://www.google.com/maps/place/%E6%98%8E%E6...
19,延平河濱公園,https://www.google.com/maps/place/%E5%BB%B6%E5...
20,迪化街,https://www.google.com/maps/place/%E8%BF%AA%E5...
21,新北大橋,https://www.google.com/maps/place/%E6%96%B0%E5...
...,...,...
986,錫安山,https://www.google.com/maps/place/%E9%8C%AB%E5...
987,朴子藝術公園,https://www.google.com/maps/place/%E6%9C%B4%E5...
988,埤頭木棉花道,https://www.google.com/maps/place/%E5%9F%A4%E9...
989,福壽山農場｜千櫻園,https://www.google.com/maps/place/%E7%A6%8F%E5...


In [24]:
# ============================================
# 個別處理
# ============================================

name = '講美鎮海社區海堤公園'
url  = get_url_for_name(name)

data = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
tag_value = data['words'].tolist()

place_detail_data = Google().get_place_info(name,url,tag_value)
place_detail_data = json.loads(place_detail_data)
print(place_detail_data)

existing_data = attraction_db.attractionInfo.find_one({
    "$or": [
        {"place_id": place_detail_data["place_id"]},
        {"id": place_detail_data["id"]}
    ]
})
print(existing_data)
if existing_data is None:
    attraction_db.attractionInfo.insert_one(place_detail_data)
    print(f"{place_detail_data['name']}：已新增")

{'name': '櫻花大道', 'google_url': 'https://www.google.com/maps/place/%E6%AB%BB%E8%8A%B1%E5%A4%A7%E9%81%93/data=!4m7!3m6!1s0x3442ae39be7123e3:0x76aeeeafb2dd3e9e!8m2!3d25.1504227!4d121.5044983!16s%2Fg%2F11b720r0xy!19sChIJ4yNxvjmuQjQRnj7dsq_urnY?authuser=0&hl=zh-TW&rclk=1', 'address': '251新北市淡水區', 'id': 'TPH_271', 'region': '新北市', 'place_id': 'ChIJAQAAABCwQjQRLbiMeGSYq48', 'google_address': '251, Taiwan, New Taipei City, Tamsui District', 'rating': 4.1, 'user_ratings_total': 602, 'lat': 25.1687611, 'lng': 121.4614806, 'photo_reference': 'AUacShhuBu-96N4eyT7AUvHd0UnpMeSsenDHbGURoDY31itmMIPoJtP9FmzE9W3pJ8qJ2EGeXF1Hsfiz2iQUHKNhHcrv9lE3ZJPAzm-p4FueibFIVjiX_xBFURJ9A_HKHZ3zCjTDq6X6HdFvk2gNGlQnbM73skls_aG7BE9udbXaucOtX-NV', 'photo_url': 'https://lh3.googleusercontent.com/places/ANJU3DsqA6Zsa-w_iOVmBmTRj4mqfePGokgc8kUzGl_k_47ZMRqyJ9K6AJk6CN5oUOtp7Z48M9uHic_l84bYzkeFZFJLNW7n9yynNrg=s1600-w1200-h1200', 'tag': ['櫻花', '櫻花季', 'ㄧ些', '健走', '登山', '北投區', '復興三路', '櫻花樹']}
{'_id': ObjectId('64eb5a10bcda7bf0f797

In [8]:
# 檢查未成功插入資料庫的景點

existing_names = set(doc['name'] for doc in attraction_db.attractionInfo.find({}, {"name": 1}))

csv_folder_path = './data/tfidf_csv/' 
csv_files = os.listdir(csv_folder_path)
new_csv_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
print(len(new_csv_files))
print(new_csv_files)

0
[]


In [ ]:
# 新增缺少的景點
wrong = []
# new_csv_files = ['三峽螢火蟲秘境']
for spot in new_csv_files:
    name = spot
    url = get_url_for_name(name)
    try:
        try:
            tfidf = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
            tag_value = tfidf['words'].tolist()
        except:
            tag_value = []
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            wrong.append({
                "name":place_detail_data['name'],
                "existing_data":existing_data['name']
            })
            print(f"{place_detail_data['name']}：未新增",existing_data['name'])
    except Exception as e:
        wrong.append({
            "name":name,
            "existing_data":e
        })
        print(f"錯誤訊息：{e}")
        continue

print(wrong)
with open("./data/warn/check.json", "w", encoding="utf8") as f:
    json.dump(wrong, f, indent=2, sort_keys=True, ensure_ascii=False)

In [7]:
# 刪除不必要的景點
for spot in new_csv_files:
    name = spot
    csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"{name}.csv 已删除")

南寮古厝.csv 已删除
月之美術館.csv 已删除
月牙灣鞦韆.csv 已删除
海濱公園.csv 已删除
石磐瀑布.csv 已删除
自由廣場牌樓.csv 已删除
草莓文化館.csv 已删除
錦和運動公園(防災公園).csv 已删除
雪霸國家公園.csv 已删除
龍騰公園.csv 已删除


In [8]:
# 隨機排序景點

list = pd.read_csv('./data/spot_list/spotsList.csv')
print(list)

random.seed(42)
random_spotsList = list.sample(frac=1,random_state=random.randint(1,10000))
print(random_spotsList)

random_spotsList.to_csv('./data/spot_list/random_spotsList.csv',encoding="utf-8-sig",index=False)


        name                                         google_url
0      嶺腳觀景台  https://www.google.com/maps/place/%E5%B6%BA%E8...
1       嶺腳瀑布  https://www.google.com/maps/place/%E5%B6%BA%E8...
2       平溪瀑布  https://www.google.com/maps/place/%E5%B9%B3%E6...
3       平溪老街  https://www.google.com/maps/place/%E5%B9%B3%E6...
4       菁桐老街  https://www.google.com/maps/place/%E8%8F%81%E6...
...      ...                                                ...
2396  八二三戰史館  https://www.google.com/maps/place/%E5%85%AB%E4...
2397   田浦風獅爺  https://www.google.com/maps/place/%E7%94%B0%E6...
2398   黃卓彬洋樓  https://www.google.com/maps/place/%E9%BB%83%E5...
2399     建功嶼  https://www.google.com/maps/chplace/%E5%BB%BA%...
2400     清遠湖  https://www.google.com/maps/place/%E6%B8%85%E9...

[2401 rows x 2 columns]
               name                                         google_url
2264     將軍區長榮里木棉花道  https://www.google.com/maps/place/%E5%B0%87%E8...
1536           精英溫泉  https://www.google.com/maps/place/%E7%B2%BE%

In [8]:
# ============================================
# 個別處理 2
# ============================================
ws_driver = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")

In [9]:
csv_folder_path = './data/set/' 
existing_names = attraction_db.attractionInfo.distinct("name")
csv_files = os.listdir(csv_folder_path)
tfidf_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
tfidf_files

['巴克禮紀念公園']

In [10]:
for spot in tfidf_files:
    name = spot
    url = get_url_for_name(name)
    try:
        dataset = pd.read_excel('./data/set/{}.xlsx'.format(name))
        tag = Word(ws_driver,pos_driver).save(dataset)
        tag_value = tag['words'].tolist()
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            print(f"{place_detail_data['name']}：未新增")
    except Exception as e:
        print(f"錯誤訊息：{e}")
        continue

巴克禮紀念公園：已新增
